In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import torch
import torch.nn as nn
import pickle
import json
import pandas as pd
import numpy as np

# Set path (adjust according to your Colab Drive path)
base_path = '/content/drive/MyDrive/Colab Notebooks/'

# Load scaler
with open(base_path + 'scaler2.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Load input feature columns
with open(base_path + 'input_columns.json', 'r') as f:
    input_columns = json.load(f)
input_dim = len(input_columns)

# Define model architecture
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.dropout = nn.Dropout(0.1)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(self.relu(self.fc2(x)))
        return self.fc3(x)

# Initialize model and load parameters
model = MLP(input_dim)
model.load_state_dict(torch.load(base_path + "mlp_model.pth"))
model.eval()

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


MLP(
  (fc1): Linear(in_features=22, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [4]:
!pip install langchain langchain-openai openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 6.5 MB/s eta 0:00:00


In [6]:
# model_tool.py
import torch
import pandas as pd
import json
import joblib
from datetime import datetime
from typing import List, Dict, Union
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.tools import Tool

# Preprocess input
def preprocess_input(data: pd.DataFrame) -> torch.Tensor:
    for col in input_columns:
        if col not in data.columns:
            data[col] = 0
    data = data[input_columns]
    scaled = scaler.transform(data)
    return torch.tensor(scaled, dtype=torch.float32)

import ast  # Safely parse string to Python object

def predict_sales(store_data: Union[str, List[Dict], Dict]) -> list:
    if isinstance(store_data, str):
        try:
            store_data = ast.literal_eval(store_data)
        except Exception as e:
            raise ValueError(f"Failed to parse input string to dictionary: {e}")

    if isinstance(store_data, dict):
        store_data = [store_data]
    elif not isinstance(store_data, list):
        raise ValueError("store_data must be a dict or list of dicts.")

    df = pd.DataFrame(store_data)
    df["Date"] = pd.to_datetime(df["Date"])
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df.fillna(0, inplace=True)
    input_tensor = preprocess_input(df)
    with torch.no_grad():
        preds = model(input_tensor).squeeze().tolist()
    return preds if isinstance(preds, list) else [preds]

# LangChain-compatible version of the promotion recommendation tool
def recommend_promotions(data: dict) -> dict:
    if isinstance(data, str):
        data = json.loads(data)
    store_list = data["store_ids"]
    date = data["date"]

    recommendation = {}
    today = pd.to_datetime(date)
    weekday = today.dayofweek + 1

    input_data = []
    for store in store_list:
        for promo in [0, 1]:
            input_data.append({
                "Store": store,
                "Date": date,
                "Promo": promo,
                "DayOfWeek": weekday,
                "Open": 1,
                # "Customers": 500  # Optionally assume default customers
            })

    df = pd.DataFrame(input_data)
    df["Date"] = pd.to_datetime(df["Date"])
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day

    input_tensor = preprocess_input(df)

    with torch.no_grad():
        preds = model(input_tensor).squeeze().tolist()

    for i, store in enumerate(store_list):
        no_promo = preds[i * 2]
        with_promo = preds[i * 2 + 1]
        recommendation[store] = "Promotion recommended" if with_promo - no_promo > 300 else "No promotion needed"

    return recommendation

# Wrap with Tool
predict_sales_tool = Tool.from_function(
    func=predict_sales,
    name="predict_sales",
    description=(
        "Predict sales based on store data. "
        "Input should be a dict or a list of dicts, each with: Store, Date (YYYY-MM-DD), Promo, DayOfWeek, Open"
    )
)

recommend_promotions_tool = Tool.from_function(
    func=recommend_promotions,
    name="recommend_promotions",
    description=(
        "Recommend promotion strategies for stores. "
        "Input is a dict with a list of store IDs like [1, 2] and a date string like '2015-07-10'"
    )
)

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Initialize Agent
agent = initialize_agent(
    tools=[predict_sales_tool, recommend_promotions_tool],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
)

# Test call (input must be structured)
# print(agent.invoke("What is the predicted sales for store 1 on 2016-07-31?"))
print(agent.invoke("Should store 1 and 2 do promotions on 2016-12-22?"))

<ipython-input-6-eac83d706c4d>:110: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
To determine if stores 1 and 2 should do promotions on 2016-12-22, I need to recommend promotion strategies for these stores on that date.

Action: recommend_promotions
Action Input: {"store_ids": [1, 2], "date": "2016-12-22"}
Observation: {1: 'Promotion recommended', 2: 'Promotion recommended'}
Thought:Both stores 1 and 2 are recommended to do promotions on 2016-12-22.

Final Answer: Yes, both store 1 and store 2 should do promotions on 2016-12-22.

> Finished chain.
{'input': 'Should store 1 and 2 do promotions on 2016-12-22?', 'output': 'Yes, both store 1 and store 2 should do promotions on 2016-12-22.'}


In [3]:
import os
os.environ["OPENAI_API_KEY"] = "secret"
